In [1]:
import torch
from torch import nn
import numpy as np
from lgg_model import *
from gensim.models import Word2Vec

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
lgg_model_path = input("请输入想使用的语言模型(无需添加后缀)：")
lgg_model_path = 'lgg_model_paths/' + lgg_model_path
lgg_model = torch.load(lgg_model_path)
lgg_model.eval()

vanilla_GRU(
  (Embedding): Embedding(709, 50)
  (GRU): GRU(50, 100, num_layers=2, batch_first=True, dropout=0.5)
  (Linear): Linear(in_features=100, out_features=709, bias=True)
)

In [3]:
word_model_path = input("请使用想使用的词汇库(无需添加后缀)：")
word_model_path = 'word_model_paths/' + word_model_path
word_model = Word2Vec.load(word_model_path)
wv = word_model.wv

In [4]:
words = input("请输入初始文本：")
del_lst = []
lst = list(words)

for i in lst:
    if i not in wv.key_to_index:
        del_lst.append(i)
for i in del_lst:
    lst.remove(i)

data = np.array([])
for i in lst:
    data = np.append(data, wv.key_to_index[i])

count = int(input("请输入想要生成的字(词)数："))

for i in lst:
    print(i, end='')

for i in range(count):
    data = np.stack((data,))
    x = torch.Tensor(data)
    x = x.to(torch.long)
    y = lgg_model(x)[0][-1]
    p = y.detach().numpy()
    p = softmax(p)

    idx = np.random.choice(np.arange(len(wv)), p=p)
    new_word = wv.index_to_key[idx]
    print(new_word, end='')

    lst.append(new_word)
    data = np.append(data, idx)

print('\nGeneration finished.')



帝高阳之兮，朕辞而不听。
倡曰：有鸟自南兮，来集北。
折疏兮瑶华，将以遗兮下女。
时不可兮再得，逾桑兮从女。
纷总总兮九州，何寿兮在予！
高回翔兮独下。
薜荔柏兮蕙，荪兮兰旌。
老冉冉兮既极，不违兮。
超回志度，行隐进兮。
低夷犹，宿北兮。
忧阳之博兮，聊逍遥以相。
及余饰之方壮兮，周流观乎上下。
灵蜷兮既留，昭昭兮未央。
鸟飞兮从文，辛夷车兮结桂旗。
望夫君兮齐，导帝之兮九。
灵衣兮屋上，水周兮堂下。
望北山而流涕兮，临流水而太息。
望瑶兮玉，将兮琼芳。
蕙蒸兮兰，桂兮椒浆。
扬枹兮鼓，疏节兮安歌，陈竽瑟兮屋来下。
绿叶兮枝，芳菲菲兮满堂。
夫悲余马怀兮，蜷不可思而不通。
清白以死直兮，固前圣之所。
悔相道之不察兮，延伫乎吾将上下兮然索。
依前圣以节中兮，凭心而历兹。
哀州土之平乐兮，余独好修以逍遥。
及少康之未家兮，留有之。
理弱而媒兮，恐导言之不固。
美要眇而失之。
欲从灵氛之吉占兮，又无良媒在其侧。
心婵媛而伤怀兮，恐若岁天谗人之嫉妒兮，被以不之名。
余既固好修以为常。
虽萎绝其亦何伤兮，进不好。
夫何极而不至兮，孰忘察余之中患。
木以结茝兮，薜荔之落。
木兰以为衣兮，春离以之。
事
Generation finished.
